<a href="https://colab.research.google.com/github/anjha1/Deep-Learning/blob/main/deep_learning_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here's a **universal deep learning template** that can be adapted for any dataset (tabular, image, text) and task (classification, regression):

```python
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from sklearn.model_selection import train_test_split
import numpy as np

# ==================== CONFIGURABLE PARAMETERS ====================
# Data parameters
DATA_PATH = "your_data_path"  # CSV, images, etc.
TARGET_COL = "target"         # For tabular data
TEST_SIZE = 0.2               # Train-test split ratio

# Model architecture
INPUT_SIZE = 784              # For MNIST: 28*28
HIDDEN_SIZES = [512, 256]     # Hidden layer dimensions
OUTPUT_SIZE = 10              # Number of classes
ACTIVATION = nn.ReLU()        # nn.Sigmoid(), nn.Tanh(), etc.
DROPOUT_RATE = 0.2            # Regularization

# Training parameters
BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 1e-3
LOSS_FN = nn.CrossEntropyLoss()  # For regression: nn.MSELoss()
OPTIMIZER = optim.Adam          # optim.SGD, optim.RMSprop, etc.

# ==================== REUSABLE COMPONENTS ====================
class CustomDataset(Dataset):
    """Universal dataset adapter"""
    def __init__(self, features, labels=None, transform=None):
        self.features = features
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        x = self.features[idx]
        if self.transform:
            x = self.transform(x)
            
        if self.labels is not None:
            y = self.labels[idx]
            return x, y
        return x

class NeuralNetwork(nn.Module):
    """Flexible MLP architecture"""
    def __init__(self):
        super().__init__()
        layers = []
        prev_size = INPUT_SIZE
        
        # Dynamically create hidden layers
        for hidden_size in HIDDEN_SIZES:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                ACTIVATION,
                nn.Dropout(DROPOUT_RATE)
            ])
            prev_size = hidden_size
            
        layers.append(nn.Linear(prev_size, OUTPUT_SIZE))
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x)

# ==================== TRAINING PIPELINE ====================
def train_model():
    # 1. Load and preprocess data (adapt based on data type)
    # Example for tabular data:
    # X, y = load_tabular_data(DATA_PATH, TARGET_COL)
    # X_train, X_test, y_train, y_test = train_test_split(...)
    
    # 2. Create datasets
    train_dataset = CustomDataset(X_train, y_train, transform=ToTensor())
    test_dataset = CustomDataset(X_test, y_test, transform=ToTensor())
    
    # 3. Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
    
    # 4. Initialize model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = NeuralNetwork().to(device)
    
    # 5. Training setup
    optimizer = OPTIMIZER(model.parameters(), lr=LEARNING_RATE)
    
    # 6. Training loop
    for epoch in range(EPOCHS):
        model.train()
        for batch in train_loader:
            X, y = batch
            X, y = X.to(device), y.to(device)
            
            # Forward pass
            pred = model(X)
            loss = LOSS_FN(pred, y)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Evaluation
        model.eval()
        with torch.no_grad():
            # Calculate metrics
            pass
            
    # 7. Save model
    torch.save(model.state_dict(), "model.pth")

# ==================== USAGE EXAMPLE ====================
if __name__ == "__main__":
    train_model()
```

### Key Features:

1. **Data Flexibility**:
   - Handles images (via `ToTensor()`), tabular data, or text (with appropriate transforms)
   - Customizable data loading through `CustomDataset` class

2. **Model Architecture**:
   - Dynamic hidden layer creation based on `HIDDEN_SIZES`
   - Configurable activation functions and dropout
   - Works for both classification and regression

3. **Training Configuration**:
   - Swappable loss functions
   - Multiple optimizer choices
   - Adjustable hyperparameters

4. **Reproducibility**:
   - Standardized training loop
   - Model saving/loading

### How to Adapt:

1. **For Different Data Types**:
   - Images: Use appropriate transforms (resize, normalize)
   - Tabular: Modify `CustomDataset` to handle DataFrames
   - Text: Add tokenization in `CustomDataset`

2. **For Different Tasks**:
   - Classification: Use `CrossEntropyLoss` and softmax output
   - Regression: Use `MSELoss` and linear output
   - Binary classification: Use `BCELoss` and sigmoid output

3. **For Advanced Architectures**:
   - Replace `NeuralNetwork` class with CNN/RNN architectures
   - Add batch normalization layers

This template provides 90% of what you need for most deep learning projects - just modify the configurable parameters and data loading logic for your specific use case.

Here's how to apply the universal template to the **CIFAR-10 dataset** (color images) with modifications clearly marked. I'll highlight all changes needed while keeping the core structure intact:

```python
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
import matplotlib.pyplot as plt

# ==================== CONFIGURATION (MODIFIED FOR CIFAR-10) ====================
# Data parameters
DATA_PATH = "./data"
TARGET_COL = None  # Not used for CIFAR-10
TEST_SIZE = 0.2    # Already split in torchvision

# Model architecture
INPUT_SIZE = 32*32*3  # CIFAR-10: 32x32 RGB images (CHANGED)
HIDDEN_SIZES = [512, 256]
OUTPUT_SIZE = 10      # CIFAR-10 has 10 classes
ACTIVATION = nn.ReLU()
DROPOUT_RATE = 0.3    # Slightly higher for images

# Training parameters
BATCH_SIZE = 128      # Increased for better GPU utilization (CHANGED)
EPOCHS = 15           # More epochs for image data (CHANGED)
LEARNING_RATE = 1e-3  
LOSS_FN = nn.CrossEntropyLoss()
OPTIMIZER = optim.Adam # Better for image data than SGD (CHANGED)

# Image transformations (NEW)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize RGB
])

# ==================== DATA LOADING (MODIFIED) ====================
# Download CIFAR-10 data (REPLACED FashionMNIST)
train_data = datasets.CIFAR10(
    root=DATA_PATH,
    train=True,
    download=True,
    transform=transform
)

test_data = datasets.CIFAR10(
    root=DATA_PATH,
    train=False,
    download=True,
    transform=transform
)

# Create dataloaders (No need for CustomDataset with torchvision)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

# ==================== MODEL ARCHITECTURE (UPDATED) ====================
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            # Simple CNN for images (NEW)
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # RGB input
            ACTIVATION,
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            ACTIVATION,
            nn.MaxPool2d(2, 2),
            nn.Flatten()  # Flatten for dense layers
        )
        
        # Dynamically created classifier
        classifier_layers = []
        prev_size = 64 * 8 * 8  # Calculated from CNN output (CHANGED)
        
        for hidden_size in HIDDEN_SIZES:
            classifier_layers.extend([
                nn.Linear(prev_size, hidden_size),
                ACTIVATION,
                nn.Dropout(DROPOUT_RATE)
            ])
            prev_size = hidden_size
            
        classifier_layers.append(nn.Linear(prev_size, OUTPUT_SIZE))
        self.classifier = nn.Sequential(*classifier_layers)
        
    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

# ==================== TRAINING PIPELINE (UNCHANGED CORE) ====================
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = LOSS_FN(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += LOSS_FN(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)
    print(f'\nTest set: Average loss: {test_loss:.4f}, '
          f'Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)\n')

# ==================== EXECUTION ====================
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = NeuralNetwork().to(device)
    optimizer = OPTIMIZER(model.parameters(), lr=LEARNING_RATE)

    for epoch in range(1, EPOCHS + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
    
    torch.save(model.state_dict(), "cifar10_model.pth")

if __name__ == "__main__":
    main()
```

### Key Adaptations Made:

1. **Data Loading**:
   - Replaced FashionMNIST with CIFAR-10
   - Added RGB normalization transform
   - Removed CustomDataset as torchvision handles it

2. **Model Architecture**:
   - Changed input size to 32×32×3 (RGB)
   - Added CNN layers before dense layers
   - Updated flattening dimension calculation

3. **Training Parameters**:
   - Increased batch size to 128
   - More epochs (15) for image data
   - Switched to Adam optimizer

4. **Normalization**:
   - Added mean/std normalization for RGB channels

### For Other Datasets:

1. **Tabular Data**:
   ```python
   # Replace data loading with:
   import pandas as pd
   from sklearn.preprocessing import StandardScaler
   
   df = pd.read_csv("your_data.csv")
   X = df.drop(TARGET_COL, axis=1).values
   y = df[TARGET_COL].values
   
   # Add scaling for numerical data
   scaler = StandardScaler()
   X = scaler.fit_transform(X)
   ```

2. **Text Data**:
   ```python
   # Add tokenization
   from transformers import AutoTokenizer
   
   tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
   texts = ["your text data here"]
   inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
   ```

The template maintains the same core structure while allowing flexible adaptation through clearly marked configuration sections.